In [1]:
# 輸送最適化問題
# 倉庫群から工場群へ部品を搬送したい。輸送費が最小となる計画を求める。
# 各倉庫からの搬出は供給可能料以下、各工場への搬入は需要量以上

In [2]:
# 変数：倉庫群から工場群への輸送量
# 目的関数：輸送コスト→最小化
# 制約条件：各倉庫からの搬出は、供給可能料以下。各工場への搬入は、需要量以上

In [13]:
import numpy as np, pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

In [14]:
# パラメータの設定
np.random.seed(1)
nw=3 # 倉庫数
nf=4 # 工場数
pr=list(product(range(nw),range(nf)))
supply=np.random.randint(30,50,nw)
demand=np.random.randint(20,40,nf)
cost=np.random.randint(10,20,(nw,nf))

In [15]:
pr

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [16]:
supply

array([35, 41, 42])

In [17]:
demand

array([28, 29, 31, 25])

In [18]:
cost

array([[10, 10, 11, 17],
       [16, 19, 12, 14],
       [15, 12, 14, 12]])

In [25]:
# pandasを使わないモデリング
m1=model_min()
v1={(i,j):LpVariable('v%d_%d'%(i,j), lowBound=0)
    for i,j in pr}
m1 += lpSum(cost[i,j]*v1[i,j] for i, j in pr)
for i in range(nw):
    m1+=lpSum(v1[i,j] for j in range(nf)) <= supply[i]
for j in range(nf):
    m1 += lpSum(v1[i,j] for i in range(nw)) >= demand[j]
m1.solve()
{k:value(x) for k, x in v1.items() if value(x) >0}

{(0, 0): 28.0,
 (0, 1): 7.0,
 (1, 2): 31.0,
 (1, 3): 5.0,
 (2, 1): 22.0,
 (2, 3): 20.0}

In [26]:
# pandasを使ったモデリング
df=pd.DataFrame([(i,j) for i, j in pr],
                columns=['倉庫','工場'])
df['輸送費']=cost.flatten()
df

,倉庫,工場,輸送費
0,0,0,10
1,0,1,10
2,0,2,11
3,0,3,17
4,1,0,16
5,1,1,19
6,1,2,12
7,1,3,14
8,2,0,15
9,2,1,12


In [28]:
m2=model_min()
addvars(df)
df

,倉庫,工場,輸送費,Var
0,0,0,10,v000013
1,0,1,10,v000014
2,0,2,11,v000015
3,0,3,17,v000016
4,1,0,16,v000017
5,1,1,19,v000018
6,1,2,12,v000019
7,1,3,14,v000020
8,2,0,15,v000021
9,2,1,12,v000022


In [31]:
m2 += lpSum(df.輸送費*df.Var)
for k,v in df.groupby('倉庫'): # groupby: 指定した列の値が同じ行をグルーピングする
    m2+=lpSum(v.Var) <= supply[k]
for k, v in df.groupby('工場'):
    m2+=lpSum(v.Var)>=demand[k]
m2.solve()
addvals(df)
df[df.Val >0]

,倉庫,工場,輸送費,Var,Val
0,0,0,10,v000013,28.0
1,0,1,10,v000014,7.0
6,1,2,12,v000019,31.0
7,1,3,14,v000020,5.0
9,2,1,12,v000022,22.0
11,2,3,12,v000024,20.0


### PuLPとpandasを組み合わせることのメリット
- 1つの変数が表の一行に対応するため、変数が分かりやすくなる
- 列名を使える
- pandasの条件式を使って数式を組み立てられる
- pandasの便利な関数を使ってモデルを作成できる
- 結果も表に追加できる
- pandasで結果を加工できる